## Registering the GBM NIFTIS

For each exam, we want to find the T1, T2, T2FLAIR, and T1C and for the first go, we're just going to skull strip using the BET program from FSL. 

We want to use the following command: 

` bet <input> <output> -S`

In [1]:
import glob
import subprocess as sub 
from subprocess import * 
import os 
from pathlib import Path
import pandas as pd

We can use the output from a prior analysis to label our images: 

In [2]:
full_label_df = pd.read_csv('/working/lupolab/julia/tcia_analysis/labels_tcia_GBM.csv')

In [3]:
full_label_df.head()

,ground_truth,image,sd
0,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,1-SAG_LOCAL-29333
1,T1,/working/lupolab/julia/tcia_analysis/datasets/...,3-AXIAL_T1-88865
2,T2,/working/lupolab/julia/tcia_analysis/datasets/...,2-AXIAL_T2-83081
3,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,4-AXIAL_T1_POST_GD-64939
4,T2,/working/lupolab/julia/tcia_analysis/datasets/...,15-3DFSE_AXIAL-04462


In [4]:
full_label_df.loc[0]['image'].split('/')[7]

'TCGA-06-0210'

In [5]:
full_label_df['patient_id'] = [x.split('/')[7] for x in full_label_df.image]

In [6]:
full_label_df['exam_id'] = [x.split('/')[8] for x in full_label_df.image]

In [7]:
full_label_df

,ground_truth,image,sd,patient_id,exam_id
0,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,1-SAG_LOCAL-29333,TCGA-06-0210,08-17-1995-19584
1,T1,/working/lupolab/julia/tcia_analysis/datasets/...,3-AXIAL_T1-88865,TCGA-06-0210,08-17-1995-19584
2,T2,/working/lupolab/julia/tcia_analysis/datasets/...,2-AXIAL_T2-83081,TCGA-06-0210,08-17-1995-19584
3,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,4-AXIAL_T1_POST_GD-64939,TCGA-06-0210,08-17-1995-19584
4,T2,/working/lupolab/julia/tcia_analysis/datasets/...,15-3DFSE_AXIAL-04462,TCGA-08-0350,12-15-1998-MSTEALTH-91569
5,OTHER,/working/lupolab/julia/tcia_analysis/datasets/...,1-TI_SAG_MR_BRAIN_WHOLE_HEAD-52088,TCGA-08-0350,12-15-1998-MSTEALTH-91569
6,T2,/working/lupolab/julia/tcia_analysis/datasets/...,5-3DFSE_AXIAL-96333,TCGA-08-0350,12-15-1998-MSTEALTH-91569
7,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,3-FMPSPGR_SAG-00857,TCGA-08-0350,12-15-1998-MSTEALTH-91569
8,T2_FLAIR,/working/lupolab/julia/tcia_analysis/datasets/...,6-FLAIR_AXIAL-64078,TCGA-08-0350,12-15-1998-MSTEALTH-91569
9,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,4-3DSPGR_AXIAL-74692,TCGA-08-0350,12-15-1998-MSTEALTH-91569


In [8]:
patient_list = full_label_df.patient_id.unique()

In [9]:
len(patient_list)

193

In [10]:
usable_patients = []
for patient in patient_list: 
    patient_df = full_label_df.loc[(full_label_df['patient_id'] == patient) & (full_label_df['ground_truth'] != "OTHER")]
#     print(patient_df)
    if len(patient_df.ground_truth.unique()) >= 4: 
        usable_patients.append(patient)

        

In [11]:
len(usable_patients)

122

In [12]:
usable_patient_df = full_label_df.loc[(full_label_df['patient_id'].isin(usable_patients)) & 
                                      (full_label_df['ground_truth'] != "OTHER")].copy()

In [13]:
usable_patient_df

,ground_truth,image,sd,patient_id,exam_id
13,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,701-BRAIN_WWO_T1_COR_POST-08207,TCGA-14-0865,06-04-1995-MRI_BRAIN_W_WOUT_CONTRAST-31488
14,T1,/working/lupolab/julia/tcia_analysis/datasets/...,301-BRAIN_WWO_T1_AX_PRE-13193,TCGA-14-0865,06-04-1995-MRI_BRAIN_W_WOUT_CONTRAST-31488
15,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,601-BRAIN_WWO_T1_AX_POST-95894,TCGA-14-0865,06-04-1995-MRI_BRAIN_W_WOUT_CONTRAST-31488
16,T1,/working/lupolab/julia/tcia_analysis/datasets/...,201-BRAIN_WWO_T1_SAG_23SL-32502,TCGA-14-0865,06-04-1995-MRI_BRAIN_W_WOUT_CONTRAST-31488
17,T2_FLAIR,/working/lupolab/julia/tcia_analysis/datasets/...,501-BRAIN_WWO_FLAIR_AX-67274,TCGA-14-0865,06-04-1995-MRI_BRAIN_W_WOUT_CONTRAST-31488
18,T2,/working/lupolab/julia/tcia_analysis/datasets/...,401-BRAIN_WWO_T2_AX_GRASE-74294,TCGA-14-0865,06-04-1995-MRI_BRAIN_W_WOUT_CONTRAST-31488
21,T2_FLAIR,/working/lupolab/julia/tcia_analysis/datasets/...,501-BRAIN_WOW_FLAIR_AX-76993,TCGA-14-0865,11-23-1995-MRI_BRAIN_W_WOUT_CONTRAST-06737
22,T1,/working/lupolab/julia/tcia_analysis/datasets/...,201-BRAIN_WOW_T1_SAG-03766,TCGA-14-0865,11-23-1995-MRI_BRAIN_W_WOUT_CONTRAST-06737
23,T2,/working/lupolab/julia/tcia_analysis/datasets/...,601-BRAIN_WOW_T2_GRASE_AX-27643,TCGA-14-0865,11-23-1995-MRI_BRAIN_W_WOUT_CONTRAST-06737
26,T1,/working/lupolab/julia/tcia_analysis/datasets/...,401-BRAIN_WOW_AX_T1_PRE-74074,TCGA-14-0865,11-23-1995-MRI_BRAIN_W_WOUT_CONTRAST-06737


In [16]:
commands = []
for idx, row in usable_patient_df.iterrows(): 
    moving_vol = row['image']
    old_vol = moving_vol.split('.n')[0]+'a'+'.nii.gz'
    output_vol = moving_vol.split('.n')[0]+'a_n'+'.nii.gz'
    command_align = 'bet '+old_vol+' '+output_vol+' -S'
    commands.append(command_align)
    

In [ ]:
commands

In [18]:
os.chdir('/working/lupolab/julia/tcia_analysis/code/')

In [19]:
x = open('skullstrip_gbms.txt', 'w')

In [20]:
x.write('{')
x.write('\n')
for command in commands: 
    x.write(command)
    x.write('\n')
x.write('}')
x.close()

In [21]:
!grid_pipeline --scriptonly skullstrip_gbms.txt
